<a href="https://colab.research.google.com/github/ajaykumar080286/DeepLearning/blob/master/cat_dog_normal_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout,Flatten,MaxPool2D, Conv2D,BatchNormalization, Activation, MaxPool2D # Added Activation

import matplotlib.pyplot as plt
import seaborn as sns

import cv2
import os
import random

In [10]:
import zipfile

# Path to your zip file
zip_path = "/content/dogs_vs_cats.zip"

# Open the zip file
zip_ref= zipfile.ZipFile(zip_path, 'r')
zip_ref.extractall("/content")
zip_ref.close()

In [12]:
mkdir = "/content/dogs_vs_cats/train"
category=['cats','dogs']

In [54]:
import os
import cv2

data = []
base_path = "/content/dogs_vs_cats/train"   # your dataset root
categories = ["cats", "dogs"]               # folder names

for i, category in enumerate(categories):
    folder_path = os.path.join(base_path, category)
    label = i   # 0 for cats, 1 for dogs

    for j in os.listdir(folder_path):
        img_path = os.path.join(folder_path, j)
        img = cv2.imread(img_path)

        if img is None:   # skip unreadable files
            continue
        img = cv2.resize(img, (150, 150))  # force same size
        data.append([img, label])

print(f"Loaded {len(data)} images")

Loaded 872 images


In [55]:
random.shuffle(data)

In [56]:
IMG_SIZE = 256
X = []
y = []

for i in data:
    X.append(i[0])
    y.append(i[1])

X = np.array(X, dtype=np.float32) / 255.0   # normalize
y = np.array(y)


In [52]:
X.shape

(872, 150, 150, 3)

In [53]:
y.shape

(872,)

In [65]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [66]:
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [67]:
model.fit(X,y,epochs=5, validation_split=0.1)

Epoch 1/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 25s 907ms/step - accuracy: 0.6261 - loss: 0.6613 - val_accuracy: 0.7614 - val_loss: 0.5454
Epoch 2/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 40s 878ms/step - accuracy: 0.6790 - loss: 0.6662 - val_accuracy: 0.7614 - val_loss: 0.6335
Epoch 3/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 23s 910ms/step - accuracy: 0.7029 - loss: 0.6549 - val_accuracy: 0.7614 - val_loss: 0.6253
Epoch 4/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 23s 921ms/step - accuracy: 0.7150 - loss: 0.6175 - val_accuracy: 0.3977 - val_loss: 0.6944
Epoch 5/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 21s 824ms/step - accuracy: 0.6454 - loss: 0.6298 - val_accuracy: 0.7614 - val_loss: 0.5048


**With Data Augmentation**

In [68]:
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [69]:
batch_size=16

In [71]:
train_dataset = ImageDataGenerator(
        rescale=1./255,
        rotation_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
        )

In [72]:
test_dataset = ImageDataGenerator(rescale=1./255)

In [74]:
train_generator = train_dataset.flow_from_directory(
        '/content/dogs_vs_cats/train',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # sin
validation_generator = test_dataset.flow_from_directory(
        '/content/dogs_vs_cats/test',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

Found 872 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [78]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [79]:
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [82]:
model.fit(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=25,
        validation_data=validation_generator,
        validation_steps=800 // batch_size)

Epoch 1/25
 55/125 ━━━━━━━━━━━━━━━━━━━━ 35s 510ms/step - accuracy: 0.7190 - loss: 0.6281

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()
/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


125/125 ━━━━━━━━━━━━━━━━━━━━ 35s 275ms/step - accuracy: 0.7126 - loss: 0.6292 - val_accuracy: 0.5000 - val_loss: 0.7413
Epoch 2/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 42s 330ms/step - accuracy: 0.7150 - loss: 0.6134 - val_accuracy: 0.5200 - val_loss: 0.7837
Epoch 3/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 34s 268ms/step - accuracy: 0.7153 - loss: 0.5840 - val_accuracy: 0.5550 - val_loss: 0.6675
Epoch 4/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 41s 278ms/step - accuracy: 0.7132 - loss: 0.5877 - val_accuracy: 0.5250 - val_loss: 0.6742
Epoch 5/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 35s 278ms/step - accuracy: 0.7167 - loss: 0.5761 - val_accuracy: 0.5138 - val_loss: 0.8574
Epoch 6/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 40s 272ms/step - accuracy: 0.7195 - loss: 0.5807 - val_accuracy: 0.6550 - val_loss: 0.6361
Epoch 7/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 42s 331ms/step - accuracy: 0.7096 - loss: 0.5466 - val_accuracy: 0.6500 - val_loss: 0.6590
Epoch 8/25
125/125 ━━━━━━━━━━━━━━━━━━━━ 41s 324ms/step - accuracy: 0.7512 - loss: 0.5421 - val

In [83]:
#https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html